# Ejercicio: Crear un DataFrame Completo con SWAPI

## Objetivo:
El objetivo de este ejercicio es que los alumnos practiquen la obtención de datos de una API REST y su posterior procesamiento para crear un DataFrame en Pandas que contenga información detallada sobre personajes de Star Wars.

## Instrucciones:

1. Obtener los datos:

- Utilizar la API de Star Wars (SWAPI) para obtener datos sobre los personajes (people) de la saga.
- La URL base para obtener la lista de personajes es: https://swapi.dev/api/people/.
- Dado que los datos están paginados, deberán recorrer todas las páginas para obtener la lista completa de personajes.

2. Crear el DataFrame:

    Crear un DataFrame que incluya las siguientes columnas con la información detallada de cada personaje:

    - id: Identificador único del personaje.
    - name: Nombre del personaje.
    - height: Altura del personaje (en centímetros).
    - mass: Peso del personaje (en kilogramos).
    - hair_color: Color de cabello.
    - skin_color: Color de piel.
    - eye_color: Color de ojos.
    - birth_year: Año de nacimiento.
    - gender: Género.
    - homeworld: Nombre del planeta de origen (obtenido mediante una petición adicional a la API).
    - species: Nombre de la especie del personaje (obtenido mediante una petición adicional si el campo species no está vacío).
    - films: Lista de nombres de películas en las que aparece el personaje (obtenido mediante peticiones adicionales).

3. Procesamiento de datos:

    - Cada entrada en el campo homeworld de los personajes es un enlace a la API, desde el cual se debe obtener el nombre del planeta.

    - El campo species contiene una lista de enlaces. Si no está vacío, se debe obtener el nombre de la especie desde la API.

    - El campo films contiene una lista de enlaces a las películas. Se debe realizar una petición por cada enlace para obtener el título de la película.

4. Entrega del ejercicio:

    - Se debe entregar un archivo .ipynb (Jupyter Notebook) con el código que obtenga los datos, procese las respuestas y construya el DataFrame completo.

    - También deben incluir en el notebook una breve descripción de cómo organizaron el proceso para obtener los datos y construir el DataFrame.

Tips:

- Utilicen la librería requests para realizar las peticiones HTTP.
- Tengan en cuenta que la API utiliza paginación, por lo que deberán iterar sobre las páginas hasta obtener todos los datos.
- Los datos relacionados (homeworld, species, films) requieren realizar peticiones adicionales, así que gestionen adecuadamente los tiempos de espera (pueden considerar utilizar la función time.sleep() para evitar sobrecargar la API).
- Si encuentran campos con datos faltantes o errores, documenten cómo lo manejan en su notebook.

In [1]:
import hashlib
import requests
import datetime
import pandas as pd
import json
import os

In [6]:
url_people = 'https://swapi.dev/api/people/'

res = requests.get(url_people).json ()
res

{'count': 82,
 'next': 'https://swapi.dev/api/people/?page=2',
 'previous': None,
 'results': [{'name': 'Luke Skywalker',
   'height': '172',
   'mass': '77',
   'hair_color': 'blond',
   'skin_color': 'fair',
   'eye_color': 'blue',
   'birth_year': '19BBY',
   'gender': 'male',
   'homeworld': 'https://swapi.dev/api/planets/1/',
   'films': ['https://swapi.dev/api/films/1/',
    'https://swapi.dev/api/films/2/',
    'https://swapi.dev/api/films/3/',
    'https://swapi.dev/api/films/6/'],
   'species': [],
   'vehicles': ['https://swapi.dev/api/vehicles/14/',
    'https://swapi.dev/api/vehicles/30/'],
   'starships': ['https://swapi.dev/api/starships/12/',
    'https://swapi.dev/api/starships/22/'],
   'created': '2014-12-09T13:50:51.644000Z',
   'edited': '2014-12-20T21:17:56.891000Z',
   'url': 'https://swapi.dev/api/people/1/'},
  {'name': 'C-3PO',
   'height': '167',
   'mass': '75',
   'hair_color': 'n/a',
   'skin_color': 'gold',
   'eye_color': 'yellow',
   'birth_year': '112BB

In [7]:
url_planets = 'https://swapi.dev/api/planets/'

res_planets = requests.get(url_planets).json ()
res_planets

{'count': 60,
 'next': 'https://swapi.dev/api/planets/?page=2',
 'previous': None,
 'results': [{'name': 'Tatooine',
   'rotation_period': '23',
   'orbital_period': '304',
   'diameter': '10465',
   'climate': 'arid',
   'gravity': '1 standard',
   'terrain': 'desert',
   'surface_water': '1',
   'population': '200000',
   'residents': ['https://swapi.dev/api/people/1/',
    'https://swapi.dev/api/people/2/',
    'https://swapi.dev/api/people/4/',
    'https://swapi.dev/api/people/6/',
    'https://swapi.dev/api/people/7/',
    'https://swapi.dev/api/people/8/',
    'https://swapi.dev/api/people/9/',
    'https://swapi.dev/api/people/11/',
    'https://swapi.dev/api/people/43/',
    'https://swapi.dev/api/people/62/'],
   'films': ['https://swapi.dev/api/films/1/',
    'https://swapi.dev/api/films/3/',
    'https://swapi.dev/api/films/4/',
    'https://swapi.dev/api/films/5/',
    'https://swapi.dev/api/films/6/'],
   'created': '2014-12-09T13:50:49.641000Z',
   'edited': '2014-12-20T

In [8]:
url_species = 'https://swapi.dev/api/species/'

res_species = requests.get(url_species).json ()
res_species

{'count': 37,
 'next': 'https://swapi.dev/api/species/?page=2',
 'previous': None,
 'results': [{'name': 'Human',
   'classification': 'mammal',
   'designation': 'sentient',
   'average_height': '180',
   'skin_colors': 'caucasian, black, asian, hispanic',
   'hair_colors': 'blonde, brown, black, red',
   'eye_colors': 'brown, blue, green, hazel, grey, amber',
   'average_lifespan': '120',
   'homeworld': 'https://swapi.dev/api/planets/9/',
   'language': 'Galactic Basic',
   'people': ['https://swapi.dev/api/people/66/',
    'https://swapi.dev/api/people/67/',
    'https://swapi.dev/api/people/68/',
    'https://swapi.dev/api/people/74/'],
   'films': ['https://swapi.dev/api/films/1/',
    'https://swapi.dev/api/films/2/',
    'https://swapi.dev/api/films/3/',
    'https://swapi.dev/api/films/4/',
    'https://swapi.dev/api/films/5/',
    'https://swapi.dev/api/films/6/'],
   'created': '2014-12-10T13:52:11.567000Z',
   'edited': '2014-12-20T21:36:42.136000Z',
   'url': 'https://swap

In [9]:
url_films = 'https://swapi.dev/api/films/'

res_films = requests.get(url_films).json ()
res_films

{'count': 6,
 'next': None,
 'previous': None,
 'results': [{'title': 'A New Hope',
   'episode_id': 4,
   'opening_crawl': "It is a period of civil war.\r\nRebel spaceships, striking\r\nfrom a hidden base, have won\r\ntheir first victory against\r\nthe evil Galactic Empire.\r\n\r\nDuring the battle, Rebel\r\nspies managed to steal secret\r\nplans to the Empire's\r\nultimate weapon, the DEATH\r\nSTAR, an armored space\r\nstation with enough power\r\nto destroy an entire planet.\r\n\r\nPursued by the Empire's\r\nsinister agents, Princess\r\nLeia races home aboard her\r\nstarship, custodian of the\r\nstolen plans that can save her\r\npeople and restore\r\nfreedom to the galaxy....",
   'director': 'George Lucas',
   'producer': 'Gary Kurtz, Rick McCallum',
   'release_date': '1977-05-25',
   'characters': ['https://swapi.dev/api/people/1/',
    'https://swapi.dev/api/people/2/',
    'https://swapi.dev/api/people/3/',
    'https://swapi.dev/api/people/4/',
    'https://swapi.dev/api/peopl

In [3]:
# Función para obtener personajes
def obtener_datos(url, params = None):
    """
    Obtiene datos de una API paginada. 
    Realiza solicitudes hasta que no haya más páginas (cuando 'next' sea None).

    Args:
        url (str): La URL base de la API.
        params (dict, opcional): Parámetros de consulta para la solicitud. Por defecto es un diccionario vacío.

    Returns:
        list: Lista con los datos combinados de todas las páginas, que son diccionarios.
    """
    if params is None:
        params = {}
        
    all_data = []
    pageN_url = url
    
    while pageN_url:
        try:
            res = requests.get(pageN_url)
            res.raise_for_status() # lanza un excepción para errores HTTP

            data = res.json()   # transforma la respuesta obtenida en un JSON


            all_data.extend(data['results'])
            pageN_url = data['next']

        except requests.exceptions.RequestException as e:
            print(f"Error al realizar la solicitud: {e}")
            break

        except KeyError:
            print("Error: Estructura de respuesta inesperada.")
            break

    return all_data

In [4]:
people = obtener_datos (url_people)

# Mostrar la cantidad de datos obtenidos y los primeros personajes
print(f"Se obtuvieron {len(people)} personajes de Star Wars.")
people[:5]  # Muestra los primeros 5 resultados

Se obtuvieron 82 personajes de Star Wars.


[{'name': 'Luke Skywalker',
  'height': '172',
  'mass': '77',
  'hair_color': 'blond',
  'skin_color': 'fair',
  'eye_color': 'blue',
  'birth_year': '19BBY',
  'gender': 'male',
  'homeworld': 'https://swapi.dev/api/planets/1/',
  'films': ['https://swapi.dev/api/films/1/',
   'https://swapi.dev/api/films/2/',
   'https://swapi.dev/api/films/3/',
   'https://swapi.dev/api/films/6/'],
  'species': [],
  'vehicles': ['https://swapi.dev/api/vehicles/14/',
   'https://swapi.dev/api/vehicles/30/'],
  'starships': ['https://swapi.dev/api/starships/12/',
   'https://swapi.dev/api/starships/22/'],
  'created': '2014-12-09T13:50:51.644000Z',
  'edited': '2014-12-20T21:17:56.891000Z',
  'url': 'https://swapi.dev/api/people/1/'},
 {'name': 'C-3PO',
  'height': '167',
  'mass': '75',
  'hair_color': 'n/a',
  'skin_color': 'gold',
  'eye_color': 'yellow',
  'birth_year': '112BBY',
  'gender': 'n/a',
  'homeworld': 'https://swapi.dev/api/planets/1/',
  'films': ['https://swapi.dev/api/films/1/',
 

In [10]:
planets = obtener_datos (url_planets)

# Mostrar la cantidad de datos obtenidos y los primeros planetas
print(f"Se obtuvieron {len(planets)} planetas.")
planets[:5]  # Muestra los primeros 5 resultados

Se obtuvieron 60 planetas.


[{'name': 'Tatooine',
  'rotation_period': '23',
  'orbital_period': '304',
  'diameter': '10465',
  'climate': 'arid',
  'gravity': '1 standard',
  'terrain': 'desert',
  'surface_water': '1',
  'population': '200000',
  'residents': ['https://swapi.dev/api/people/1/',
   'https://swapi.dev/api/people/2/',
   'https://swapi.dev/api/people/4/',
   'https://swapi.dev/api/people/6/',
   'https://swapi.dev/api/people/7/',
   'https://swapi.dev/api/people/8/',
   'https://swapi.dev/api/people/9/',
   'https://swapi.dev/api/people/11/',
   'https://swapi.dev/api/people/43/',
   'https://swapi.dev/api/people/62/'],
  'films': ['https://swapi.dev/api/films/1/',
   'https://swapi.dev/api/films/3/',
   'https://swapi.dev/api/films/4/',
   'https://swapi.dev/api/films/5/',
   'https://swapi.dev/api/films/6/'],
  'created': '2014-12-09T13:50:49.641000Z',
  'edited': '2014-12-20T20:58:18.411000Z',
  'url': 'https://swapi.dev/api/planets/1/'},
 {'name': 'Alderaan',
  'rotation_period': '24',
  'orb

In [11]:
species = obtener_datos (url_species)

# Mostrar la cantidad de datos obtenidos y las primeras especies
print(f"Se obtuvieron {len(species)} especies.")
species[:5]  # Muestra los primeros 5 resultados

Se obtuvieron 37 especies.


[{'name': 'Human',
  'classification': 'mammal',
  'designation': 'sentient',
  'average_height': '180',
  'skin_colors': 'caucasian, black, asian, hispanic',
  'hair_colors': 'blonde, brown, black, red',
  'eye_colors': 'brown, blue, green, hazel, grey, amber',
  'average_lifespan': '120',
  'homeworld': 'https://swapi.dev/api/planets/9/',
  'language': 'Galactic Basic',
  'people': ['https://swapi.dev/api/people/66/',
   'https://swapi.dev/api/people/67/',
   'https://swapi.dev/api/people/68/',
   'https://swapi.dev/api/people/74/'],
  'films': ['https://swapi.dev/api/films/1/',
   'https://swapi.dev/api/films/2/',
   'https://swapi.dev/api/films/3/',
   'https://swapi.dev/api/films/4/',
   'https://swapi.dev/api/films/5/',
   'https://swapi.dev/api/films/6/'],
  'created': '2014-12-10T13:52:11.567000Z',
  'edited': '2014-12-20T21:36:42.136000Z',
  'url': 'https://swapi.dev/api/species/1/'},
 {'name': 'Droid',
  'classification': 'artificial',
  'designation': 'sentient',
  'average_

In [13]:
films = obtener_datos (url_films)

# Mostrar la cantidad de datos obtenidos y las primeras películas
print(f"Se obtuvieron {len(films)} films.")
films[:5]  # Muestra los primeros 5 resultados

Se obtuvieron 6 films.


[{'title': 'A New Hope',
  'episode_id': 4,
  'opening_crawl': "It is a period of civil war.\r\nRebel spaceships, striking\r\nfrom a hidden base, have won\r\ntheir first victory against\r\nthe evil Galactic Empire.\r\n\r\nDuring the battle, Rebel\r\nspies managed to steal secret\r\nplans to the Empire's\r\nultimate weapon, the DEATH\r\nSTAR, an armored space\r\nstation with enough power\r\nto destroy an entire planet.\r\n\r\nPursued by the Empire's\r\nsinister agents, Princess\r\nLeia races home aboard her\r\nstarship, custodian of the\r\nstolen plans that can save her\r\npeople and restore\r\nfreedom to the galaxy....",
  'director': 'George Lucas',
  'producer': 'Gary Kurtz, Rick McCallum',
  'release_date': '1977-05-25',
  'characters': ['https://swapi.dev/api/people/1/',
   'https://swapi.dev/api/people/2/',
   'https://swapi.dev/api/people/3/',
   'https://swapi.dev/api/people/4/',
   'https://swapi.dev/api/people/5/',
   'https://swapi.dev/api/people/6/',
   'https://swapi.dev/ap

__________
esto guarda las listas de diccionarios obtenidas en json, para trabajar si la url no está accesible, es decir, en un avión, o dejan de estar los datos accesibles

In [ ]:
# Guardar las listas como un archivo JSON
with open("people.json", "w", encoding="utf-8") as file:
    json.dump(people, file, ensure_ascii=False, indent=4)

    # Establecer el archivo como solo lectura
os.chmod("people.json", 0o444)  # Permisos de solo lectura
print(f"Datos guardados en people.json, {len(people)} personajes. Solo lectura")

with open("planets.json", "w", encoding="utf-8") as file:
    json.dump(planets, file, ensure_ascii=False, indent=4)
os.chmod("planets.json", 0o444)  # Permisos de solo lectura
print(f"Datos guardados en planets.json, {len(planets)} planetas. Solo lectura")

with open("species.json", "w", encoding="utf-8") as file:
    json.dump(species, file, ensure_ascii=False, indent=4)
os.chmod("species.json", 0o444)  # Permisos de solo lectura
print(f"Datos guardados en species.json, {len(species)} species. Solo lectura")

with open("films.json", "w", encoding="utf-8") as file:
    json.dump(films, file, ensure_ascii=False, indent=4)
os.chmod("films.json", 0o444)  # Permisos de solo lectura
print(f"Datos guardados en films.json, {len(films)} films. Solo lectura")

Y para leer todos los datos guardados

In [ ]:
with open("people.json", "r", encoding="utf-8") as file:
    people = json.load(file)

people[:2]  # Imprime los dos primeros registros

with open("planets.json", "r", encoding="utf-8") as file:
    planets = json.load(file)

planets[:2]

with open("species.json", "r", encoding="utf-8") as file:
    species = json.load(file)

species[:2]

with open("films.json", "r", encoding="utf-8") as file:
    films = json.load(file)

films[:2]

_______________________
ahora hago una prueba del montaje del Dataframe provisional. 

In [15]:
resultados_personajes = res ['results']
resultados_personajes

[{'name': 'Luke Skywalker',
  'height': '172',
  'mass': '77',
  'hair_color': 'blond',
  'skin_color': 'fair',
  'eye_color': 'blue',
  'birth_year': '19BBY',
  'gender': 'male',
  'homeworld': 'https://swapi.dev/api/planets/1/',
  'films': ['https://swapi.dev/api/films/1/',
   'https://swapi.dev/api/films/2/',
   'https://swapi.dev/api/films/3/',
   'https://swapi.dev/api/films/6/'],
  'species': [],
  'vehicles': ['https://swapi.dev/api/vehicles/14/',
   'https://swapi.dev/api/vehicles/30/'],
  'starships': ['https://swapi.dev/api/starships/12/',
   'https://swapi.dev/api/starships/22/'],
  'created': '2014-12-09T13:50:51.644000Z',
  'edited': '2014-12-20T21:17:56.891000Z',
  'url': 'https://swapi.dev/api/people/1/'},
 {'name': 'C-3PO',
  'height': '167',
  'mass': '75',
  'hair_color': 'n/a',
  'skin_color': 'gold',
  'eye_color': 'yellow',
  'birth_year': '112BBY',
  'gender': 'n/a',
  'homeworld': 'https://swapi.dev/api/planets/1/',
  'films': ['https://swapi.dev/api/films/1/',
 

In [17]:
personajes_df = pd.DataFrame(resultados_personajes)
personajes_df

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,https://swapi.dev/api/people/2/
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,https://swapi.dev/api/planets/8/,"[https://swapi.dev/api/films/1/, https://swapi...",[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,https://swapi.dev/api/people/3/
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[https://swapi.dev/api/starships/13/],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,https://swapi.dev/api/people/4/
4,Leia Organa,150,49,brown,light,brown,19BBY,female,https://swapi.dev/api/planets/2/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[https://swapi.dev/api/vehicles/30/],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,https://swapi.dev/api/people/5/
5,Owen Lars,178,120,"brown, grey",light,blue,52BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[],2014-12-10T15:52:14.024000Z,2014-12-20T21:17:50.317000Z,https://swapi.dev/api/people/6/
6,Beru Whitesun lars,165,75,brown,light,blue,47BBY,female,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[],[],2014-12-10T15:53:41.121000Z,2014-12-20T21:17:50.319000Z,https://swapi.dev/api/people/7/
7,R5-D4,97,32,n/a,"white, red",red,unknown,n/a,https://swapi.dev/api/planets/1/,[https://swapi.dev/api/films/1/],[https://swapi.dev/api/species/2/],[],[],2014-12-10T15:57:50.959000Z,2014-12-20T21:17:50.321000Z,https://swapi.dev/api/people/8/
8,Biggs Darklighter,183,84,black,light,brown,24BBY,male,https://swapi.dev/api/planets/1/,[https://swapi.dev/api/films/1/],[],[],[https://swapi.dev/api/starships/12/],2014-12-10T15:59:50.509000Z,2014-12-20T21:17:50.323000Z,https://swapi.dev/api/people/9/
9,Obi-Wan Kenobi,182,77,"auburn, white",fair,blue-gray,57BBY,male,https://swapi.dev/api/planets/20/,"[https://swapi.dev/api/films/1/, https://swapi...",[],[https://swapi.dev/api/vehicles/38/],"[https://swapi.dev/api/starships/48/, https://...",2014-12-10T16:16:29.192000Z,2014-12-20T21:17:50.325000Z,https://swapi.dev/api/people/10/


In [18]:
# Guardar el resultado de personajes en un archivo JSON (ojo, de momento, solo está la página 1, con los 10 primeros)
personajes_df.to_json("personajes_p1.json", indent=4, index = True)

print("El archivo JSON se ha guardado como 'personajes_p1.json'.")

El archivo JSON se ha guardado como 'personajes_p1.json'.


In [19]:
# respuesta['results']
res['results']

[{'name': 'Luke Skywalker',
  'height': '172',
  'mass': '77',
  'hair_color': 'blond',
  'skin_color': 'fair',
  'eye_color': 'blue',
  'birth_year': '19BBY',
  'gender': 'male',
  'homeworld': 'https://swapi.dev/api/planets/1/',
  'films': ['https://swapi.dev/api/films/1/',
   'https://swapi.dev/api/films/2/',
   'https://swapi.dev/api/films/3/',
   'https://swapi.dev/api/films/6/'],
  'species': [],
  'vehicles': ['https://swapi.dev/api/vehicles/14/',
   'https://swapi.dev/api/vehicles/30/'],
  'starships': ['https://swapi.dev/api/starships/12/',
   'https://swapi.dev/api/starships/22/'],
  'created': '2014-12-09T13:50:51.644000Z',
  'edited': '2014-12-20T21:17:56.891000Z',
  'url': 'https://swapi.dev/api/people/1/'},
 {'name': 'C-3PO',
  'height': '167',
  'mass': '75',
  'hair_color': 'n/a',
  'skin_color': 'gold',
  'eye_color': 'yellow',
  'birth_year': '112BBY',
  'gender': 'n/a',
  'homeworld': 'https://swapi.dev/api/planets/1/',
  'films': ['https://swapi.dev/api/films/1/',
 

- id: Identificador único del personaje.
- name: Nombre del personaje.
- height: Altura del personaje (en centímetros).
- mass: Peso del personaje (en kilogramos).
- hair_color: Color de cabello.
- skin_color: Color de piel.
- eye_color: Color de ojos.
- birth_year: Año de nacimiento.
- gender: Género.
- homeworld: Nombre del planeta de origen (obtenido mediante una petición adicional a la API).
- species: Nombre de la especie del personaje (obtenido mediante una petición adicional si el campo species no está vacío).
- films: Lista de nombres de películas en las que aparece el personaje (obtenido mediante peticiones adicionales).

In [20]:
# hago una prueba de que voy por el buen camino, antes de contruir todas las listas de datos
print(res['results'][1]['name'])
res['results'][1].get('name')

C-3PO


'C-3PO'

In [21]:
people [0].get('homeworld')

'https://swapi.dev/api/planets/1/'

In [22]:
planets['url' == 'https://swapi.dev/api/planets/1/'].get('name')

'Tatooine'

In [23]:
planets['url' == people [0].get('homeworld')].get('name')

'Tatooine'

In [38]:
result = next((item for item in planets if item['url'] == people [24].get('homeworld')), None)

if result:
    result.get('name')
else:
    print("No se encontró el planeta.")

In [39]:
people [1].get('species')

['https://swapi.dev/api/species/2/']

In [40]:
species['url' == 'https://swapi.dev/api/species/3/'].get('name')

AttributeError: 'str' object has no attribute 'get'

In [41]:
species['url' == people [1].get('species')].get('name')

AttributeError: 'str' object has no attribute 'get'

In [28]:
specie_name = {specie['url'] : specie['name'] for specie in species}
specie_name

{'https://swapi.dev/api/species/1/': 'Human',
 'https://swapi.dev/api/species/2/': 'Droid',
 'https://swapi.dev/api/species/3/': 'Wookie',
 'https://swapi.dev/api/species/4/': 'Rodian',
 'https://swapi.dev/api/species/5/': 'Hutt',
 'https://swapi.dev/api/species/6/': "Yoda's species",
 'https://swapi.dev/api/species/7/': 'Trandoshan',
 'https://swapi.dev/api/species/8/': 'Mon Calamari',
 'https://swapi.dev/api/species/9/': 'Ewok',
 'https://swapi.dev/api/species/10/': 'Sullustan',
 'https://swapi.dev/api/species/11/': 'Neimodian',
 'https://swapi.dev/api/species/12/': 'Gungan',
 'https://swapi.dev/api/species/13/': 'Toydarian',
 'https://swapi.dev/api/species/14/': 'Dug',
 'https://swapi.dev/api/species/15/': "Twi'lek",
 'https://swapi.dev/api/species/16/': 'Aleena',
 'https://swapi.dev/api/species/17/': 'Vulptereen',
 'https://swapi.dev/api/species/18/': 'Xexto',
 'https://swapi.dev/api/species/19/': 'Toong',
 'https://swapi.dev/api/species/20/': 'Cerean',
 'https://swapi.dev/api/spec

In [42]:
film_name = {film['url'] : film['title'] for film in films}
film_name

TypeError: string indices must be integers, not 'str'

In [32]:
lista_id = []
lista_name = []
lista_height = []
lista_mass = []
lista_hair_color = []
lista_skin_color = []
lista_eye_color = []
lista_birth_year = []
lista_gender = []
lista_homeworld = []
lista_species = []
lista_films = []

#Creación de diccionarios sencillos con cada url y el dato que nos piden de ese planeta 
planet_name = {planet['url'] : planet['name'] for planet in planets}

#idem para las especies
specie_name = {specie['url'] : specie['name'] for specie in species}

#idem para las películas
film_name = {film['url'] : film['title'] for film in films}

for i in range(len(people)):
    personaje = people[i]
    id = i+1
    name = personaje.get('name')
    height = personaje.get('height')  
    mass = personaje.get('mass')
    hair_color = personaje.get('hair_color')
    skin_color = personaje.get('skin_color')
    eye_color = personaje.get('eye_color')
    birth_year = personaje.get('birth_year')
    gender = personaje.get('gender')

    #homeworld = planets['url' == people [i].get('homeworld')].get('name')
    homeworld = planet_name.get(i['homeworld'],'Unknown')
    
    species = personaje.get('species')
    films = personaje.get('films')
    lista_id.append(id)
    lista_name.append(name)
    lista_height.append(height)
    lista_mass.append(mass)
    lista_hair_color.append(hair_color)
    lista_skin_color.append(skin_color)
    lista_eye_color.append(eye_color)
    lista_birth_year.append(birth_year)
    lista_gender.append(gender)
    #lista_homeworld.append(homeworld)
    lista_species.append(species)
    lista_films.append(films)




TypeError: string indices must be integers, not 'str'

In [ ]:
dict = {'id': lista_id,  
        'name' : lista_name, 'height' : lista_height, 'mass' : lista_mass, 
        'hair_color' : lista_hair_color, 'skin_color' : lista_skin_color, 'eye_color' : lista_eye_color, 
        'birth_year' : lista_birth_year, 'gender' : lista_gender, 'homeworld' : lista_homeworld, 
        'species' : lista_species, 'films' : lista_films}
dict

In [ ]:
SW_personajes_info = pd.DataFrame (dict, index = 'id')
SW_personajes_info